In [1]:
from google.cloud import storage,vision
import pandas as pd
import validators

In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credintial/GCP_CRED.json"

In [3]:
def objects_detection(path):
    n=[]
    
    if validators.url(path):
        client = vision.ImageAnnotatorClient()
        image = vision.Image()
        image.source.image_uri = path
    else:
        try:
            client = vision.ImageAnnotatorClient()
            with open(path, "rb") as image_file:
                content = image_file.read()
            image = vision.Image(content=content)
        except:
            return 'Invalid Url'

    objects = client.object_localization(image=image).localized_object_annotations
    for obj in objects:
        naam=obj.name
        n.append(naam)
    return n

    

In [6]:
objects_detection("https://t4.ftcdn.net/jpg/02/09/48/93/360_F_209489330_JaX2cOjv0iIwjyPA29Yf7Cb52hq78K9H.jpg")

['Banana', 'Orange', 'Orange']

In [7]:
storage_client = storage.Client()
bucket = storage_client.get_bucket('image-website-bucket')
blob = bucket.blob('image-data/DATA2.csv')
path = "gs://image-website-bucket/image-data/DATA2.csv"
df = pd.read_csv(path)

In [8]:
Cname= [item.lower() for item in Cname]
# print(Cname)
final_df = df[df['Pname'].isin(Cname)]
# final_df.head()
about_prd= final_df['About']


NameError: name 'Cname' is not defined

In [4]:
def product_description(img):
    Cname=objects_detection(img)
    if Cname=="Invalid Url":
        return Cname
    else:
        Cname= [item.lower() for item in Cname]
        # print(Cname)
        final_df = df[df['Pname'].isin(Cname)]
        # final_df.head()
        about_prd= final_df['About']
            
        return list(about_prd)


In [64]:
l=product_description("https://t4.ftcdn.net/jpg/02/09/48/93/360_F_209489330_JaX2cOjv0iIwjyPA29Yf7Cb52hq78K9H.jpg")
print(l)

['Curved, yellow fruit with a creamy texture and sweet taste, packed with essential nutrients and energy.', 'The orange originated in a region encompassing Southern China, Northeast India, and Myanmar;']


In [5]:
def detect_web(uri):
    """Detects web annotations given an image."""
    url=[]
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri

    response = client.web_detection(image=image)
    annotations = response.web_detection

    for image in annotations.visually_similar_images:
        url.append(image.url)
    return url



In [6]:
def upload_to_csv(names, urls, filename):
    try:
        existing_df = pd.read_csv(filename)
    except FileNotFoundError:
        existing_df = pd.DataFrame({'Pname': [], 'URL': []})

    min_length = min(len(names), len(urls))
    new_data = {'Pname': names[:min_length], 'URL': urls[:min_length]}
    new_df = pd.DataFrame(new_data)

    combined_df = pd.concat([existing_df, new_df], ignore_index=True)
    combined_df.to_csv(filename, index=False)

In [42]:
def label_detection(path):
    labels = []
    
    if validators.url(path):
        client = vision.ImageAnnotatorClient()
        image = vision.Image()
        image.source.image_uri = path
    else:
        try:
            client = vision.ImageAnnotatorClient()
            with open(path, "rb") as image_file:
                content = image_file.read()
            image = vision.Image(content=content)
        except:
            return 'Invalid Url'

    response = client.label_detection(image=image)
    for label in response.label_annotations:
        labels.append(label.description)
    return labels


def logo_detection(path):
    logos = []
    
    if validators.url(path):
        client = vision.ImageAnnotatorClient()
        image = vision.Image()
        image.source.image_uri = path
    else:
        try:
            client = vision.ImageAnnotatorClient()
            with open(path, "rb") as image_file:
                content = image_file.read()
            image = vision.Image(content=content)
        except:
            return 'Invalid Url'

    response = client.logo_detection(image=image)
    for logo in response.logo_annotations:
        logos.append(logo.description)
    return logos


In [23]:
path="https://teaklab.com/wp-content/uploads/2019/08/5adefd5eNa9b4ddfb.jpg"
got_url=detect_web(path)
# n1=logo_detection(path)
# n1=[item.lower() for item in n1]
# n2=label_detection(path)
# n2=[item.lower() for item in n2]
obj_name=objects_detection(path)
obj_small=[item.lower() for item in obj_name]
# new_list=n1+n2+obj_small
filename="DATA4.csv"
upload_to_csv(obj_small,got_url,filename)
